<a href="https://colab.research.google.com/github/mb8655/Python/blob/main/Storing_from_Pandas_to_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading a Dataset to a Database

In [16]:
!pip3 install -U -q PyMySQL sqlalchemy
import pandas as pd
from sqlalchemy import create_engine

In [17]:
#Downloading Data and Putting in a Dataframe
!curl 'https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD' -o restaurants.csv

# Read the CSV file
df = pd.read_csv('restaurants.csv')

df

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  116M    0  116M    0     0  6507k      0 --:--:--  0:00:18 --:--:-- 6811k


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,50162033,WINGSTOP,Manhattan,2665,BROADWAY,10025.0,2125055861,NaN,01/01/1900,NaN,...,NaN,40.798078,-73.969293,107.0,6.0,18700.0,1056504.0,1.018730e+09,MN12,NaN
1,50166191,GUAJURA ADULT DAY CARE INC,Bronx,1776,BOSTON ROAD,10460.0,9178171273,NaN,01/01/1900,NaN,...,NaN,40.837932,-73.886811,203.0,17.0,16100.0,2000000.0,2.029910e+09,BX75,NaN
2,50138579,"KKOMA GIMBAB, INC.",Queens,15051,NORTHERN BLVD,11354.0,3476330566,NaN,01/01/1900,NaN,...,NaN,40.765136,-73.813666,407.0,20.0,115700.0,4113286.0,4.050090e+09,QN51,NaN
3,50095998,HO HO GARDEN,Queens,9306,LIBERTY AVE,11417.0,7183226666,Chinese,08/30/2024,Establishment re-opened by DOHMH.,...,Cycle Inspection / Reopening Inspection,40.680189,-73.845693,410.0,32.0,5400.0,4190993.0,4.091620e+09,QN56,NaN
4,50158848,J LEVEL LLC,Manhattan,130,EAST 57 STREET,10022.0,9173882963,NaN,01/01/1900,NaN,...,NaN,40.761062,-73.969487,105.0,4.0,11203.0,1036890.0,1.013110e+09,MN19,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274383,50119784,OLIVE DELI & CAFE,Queens,8239,164TH ST,11432.0,3474847348,American,10/10/2023,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.718233,-73.803679,408.0,24.0,126700.0,4152343.0,4.070380e+09,QN38,NaN
274384,50106335,MASALA KING,Manhattan,64,EAST 34 STREET,10016.0,6467670230,Indian,04/23/2024,Establishment Closed by DOHMH. Violations were...,...,Cycle Inspection / Re-inspection,40.747467,-73.982294,105.0,4.0,7400.0,1078717.0,1.008630e+09,MN17,NaN
274385,41648187,EL NUEVO SANDY RESTAURANT,Manhattan,2261,2 AVENUE,10035.0,2123488654,Latin American,07/01/2024,Violations were cited in the following area(s).,...,Cycle Inspection / Re-inspection,40.797101,-73.937708,111.0,8.0,18800.0,1052582.0,1.016660e+09,MN34,NaN
274386,41138984,HOTEL PIERRE-BANQUET KITCHEN,Manhattan,2,EAST 61 STREET,10065.0,2128388000,American,09/22/2022,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.765402,-73.971864,108.0,4.0,11401.0,1040809.0,1.013750e+09,MN40,NaN


In [18]:
# Adding underscores in all column names
cols = df.columns
cols = cols.map(lambda x: x.replace(' ', '_').upper())
df.columns = cols

df

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE_DESCRIPTION,INSPECTION_DATE,ACTION,...,INSPECTION_TYPE,LATITUDE,LONGITUDE,COMMUNITY_BOARD,COUNCIL_DISTRICT,CENSUS_TRACT,BIN,BBL,NTA,LOCATION_POINT1
0,50162033,WINGSTOP,Manhattan,2665,BROADWAY,10025.0,2125055861,NaN,01/01/1900,NaN,...,NaN,40.798078,-73.969293,107.0,6.0,18700.0,1056504.0,1.018730e+09,MN12,NaN
1,50166191,GUAJURA ADULT DAY CARE INC,Bronx,1776,BOSTON ROAD,10460.0,9178171273,NaN,01/01/1900,NaN,...,NaN,40.837932,-73.886811,203.0,17.0,16100.0,2000000.0,2.029910e+09,BX75,NaN
2,50138579,"KKOMA GIMBAB, INC.",Queens,15051,NORTHERN BLVD,11354.0,3476330566,NaN,01/01/1900,NaN,...,NaN,40.765136,-73.813666,407.0,20.0,115700.0,4113286.0,4.050090e+09,QN51,NaN
3,50095998,HO HO GARDEN,Queens,9306,LIBERTY AVE,11417.0,7183226666,Chinese,08/30/2024,Establishment re-opened by DOHMH.,...,Cycle Inspection / Reopening Inspection,40.680189,-73.845693,410.0,32.0,5400.0,4190993.0,4.091620e+09,QN56,NaN
4,50158848,J LEVEL LLC,Manhattan,130,EAST 57 STREET,10022.0,9173882963,NaN,01/01/1900,NaN,...,NaN,40.761062,-73.969487,105.0,4.0,11203.0,1036890.0,1.013110e+09,MN19,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274383,50119784,OLIVE DELI & CAFE,Queens,8239,164TH ST,11432.0,3474847348,American,10/10/2023,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.718233,-73.803679,408.0,24.0,126700.0,4152343.0,4.070380e+09,QN38,NaN
274384,50106335,MASALA KING,Manhattan,64,EAST 34 STREET,10016.0,6467670230,Indian,04/23/2024,Establishment Closed by DOHMH. Violations were...,...,Cycle Inspection / Re-inspection,40.747467,-73.982294,105.0,4.0,7400.0,1078717.0,1.008630e+09,MN17,NaN
274385,41648187,EL NUEVO SANDY RESTAURANT,Manhattan,2261,2 AVENUE,10035.0,2123488654,Latin American,07/01/2024,Violations were cited in the following area(s).,...,Cycle Inspection / Re-inspection,40.797101,-73.937708,111.0,8.0,18800.0,1052582.0,1.016660e+09,MN34,NaN
274386,41138984,HOTEL PIERRE-BANQUET KITCHEN,Manhattan,2,EAST 61 STREET,10065.0,2128388000,American,09/22/2022,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.765402,-73.971864,108.0,4.0,11401.0,1040809.0,1.013750e+09,MN40,NaN


In [19]:
# view the colum names

cols

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE_DESCRIPTION', 'INSPECTION_DATE', 'ACTION', 'VIOLATION_CODE',
       'VIOLATION_DESCRIPTION', 'CRITICAL_FLAG', 'SCORE', 'GRADE',
       'GRADE_DATE', 'RECORD_DATE', 'INSPECTION_TYPE', 'LATITUDE', 'LONGITUDE',
       'COMMUNITY_BOARD', 'COUNCIL_DISTRICT', 'CENSUS_TRACT', 'BIN', 'BBL',
       'NTA', 'LOCATION_POINT1'],
      dtype='object')

In [20]:
# Some bookkeeping regarding datatypes
df["INSPECTION_DATE"] = pd.to_datetime(df["INSPECTION_DATE"], format="%m/%d/%Y")
df["SCORE"] = pd.to_numeric(df["SCORE"])

# Delete useless columns
df = df.drop(['GRADE_DATE', 'RECORD_DATE', 'LOCATION_POINT1'], axis='columns')
df

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE_DESCRIPTION,INSPECTION_DATE,ACTION,...,GRADE,INSPECTION_TYPE,LATITUDE,LONGITUDE,COMMUNITY_BOARD,COUNCIL_DISTRICT,CENSUS_TRACT,BIN,BBL,NTA
0,50162033,WINGSTOP,Manhattan,2665,BROADWAY,10025.0,2125055861,NaN,1900-01-01,NaN,...,NaN,NaN,40.798078,-73.969293,107.0,6.0,18700.0,1056504.0,1.018730e+09,MN12
1,50166191,GUAJURA ADULT DAY CARE INC,Bronx,1776,BOSTON ROAD,10460.0,9178171273,NaN,1900-01-01,NaN,...,NaN,NaN,40.837932,-73.886811,203.0,17.0,16100.0,2000000.0,2.029910e+09,BX75
2,50138579,"KKOMA GIMBAB, INC.",Queens,15051,NORTHERN BLVD,11354.0,3476330566,NaN,1900-01-01,NaN,...,NaN,NaN,40.765136,-73.813666,407.0,20.0,115700.0,4113286.0,4.050090e+09,QN51
3,50095998,HO HO GARDEN,Queens,9306,LIBERTY AVE,11417.0,7183226666,Chinese,2024-08-30,Establishment re-opened by DOHMH.,...,Z,Cycle Inspection / Reopening Inspection,40.680189,-73.845693,410.0,32.0,5400.0,4190993.0,4.091620e+09,QN56
4,50158848,J LEVEL LLC,Manhattan,130,EAST 57 STREET,10022.0,9173882963,NaN,1900-01-01,NaN,...,NaN,NaN,40.761062,-73.969487,105.0,4.0,11203.0,1036890.0,1.013110e+09,MN19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274383,50119784,OLIVE DELI & CAFE,Queens,8239,164TH ST,11432.0,3474847348,American,2023-10-10,Violations were cited in the following area(s).,...,NaN,Cycle Inspection / Initial Inspection,40.718233,-73.803679,408.0,24.0,126700.0,4152343.0,4.070380e+09,QN38
274384,50106335,MASALA KING,Manhattan,64,EAST 34 STREET,10016.0,6467670230,Indian,2024-04-23,Establishment Closed by DOHMH. Violations were...,...,NaN,Cycle Inspection / Re-inspection,40.747467,-73.982294,105.0,4.0,7400.0,1078717.0,1.008630e+09,MN17
274385,41648187,EL NUEVO SANDY RESTAURANT,Manhattan,2261,2 AVENUE,10035.0,2123488654,Latin American,2024-07-01,Violations were cited in the following area(s).,...,Z,Cycle Inspection / Re-inspection,40.797101,-73.937708,111.0,8.0,18800.0,1052582.0,1.016660e+09,MN34
274386,41138984,HOTEL PIERRE-BANQUET KITCHEN,Manhattan,2,EAST 61 STREET,10065.0,2128388000,American,2022-09-22,Violations were cited in the following area(s).,...,NaN,Cycle Inspection / Initial Inspection,40.765402,-73.971864,108.0,4.0,11401.0,1040809.0,1.013750e+09,MN40


#create connection to MySQL


In [21]:
import os
from sqlalchemy import create_engine, text

# Connect to the MySQL, and use the "public" database
conn_string = 'mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8'.format(
    user     = 'student',
    password = 'dwdstudent2025',
    host     = 'db.ipeirotis.org',
    port     = 3306,
    encoding = 'utf-8',
    db = 'public'
)
engine = create_engine(conn_string)

Create the table for storing the data
Although we can let Pandas create the table automatically, the choice of data types of not always great. It is better to manually define the data types for the database.

In [22]:
# Report the maximum string lengths for
# the textual attributes. Useful when creating
# a table in SQL.
for c in df.columns.values:
  if df.dtypes[c] == 'object':
    print(c, df[c].str.len().max())

DBA 100.0
BORO 13
BUILDING 10.0
STREET 40.0
PHONE 12.0
CUISINE_DESCRIPTION 30.0
ACTION 130.0
VIOLATION_CODE 5.0
VIOLATION_DESCRIPTION 952.0
CRITICAL_FLAG 14
GRADE 1.0
INSPECTION_TYPE 59.0
NTA 4.0


In [23]:
# To avoid conflicts between people writing in the same database, we add a random suffix in the tables
# We only create the variable once while running the notebook
import uuid
if "suffix" not in globals():
    suffix = str(uuid.uuid4())[:8]
print(suffix)

5ae1294b


In [24]:
# Define the MySQL database that we will use to store the table
db_name = "public"

# The name f the table that we will use
table_name = f"inspections_{suffix}"

In [25]:
drop_table_sql = f'''
DROP TABLE IF EXISTS {db_name}.{table_name};
'''

with engine.connect() as con:
    con.execute(text(drop_table_sql))


create_table_sql = f'''
CREATE TABLE IF NOT EXISTS {db_name}.{table_name} (
    CAMIS CHAR(8),
    DBA VARCHAR(100),
    BUILDING VARCHAR(10),
    STREET VARCHAR(40),
    ZIPCODE CHAR(5),
    BORO VARCHAR(15),
    PHONE CHAR(12),
    CUISINE_DESCRIPTION VARCHAR(30),
    LATITUDE FLOAT,
    LONGITUDE FLOAT,
    COMMUNITY_BOARD CHAR(3),
    COUNCIL_DISTRICT CHAR(2),
    CENSUS_TRACT CHAR(6),
    BIN CHAR(7),
    BBL CHAR(10),
    NTA CHAR(4),
    INSPECTION_DATE DATETIME,
    ACTION VARCHAR(130),
    GRADE CHAR(1),
    INSPECTION_TYPE VARCHAR(60),
    VIOLATION_CODE VARCHAR(10),
    VIOLATION_DESCRIPTION VARCHAR(1000),
    CRITICAL_FLAG VARCHAR(15),
    SCORE SMALLINT
)  ENGINE=INNODB DEFAULT CHARSET=UTF8MB4;
'''

with engine.connect() as con:
    con.execute(text(create_table_sql))

In [26]:

with engine.connect() as con:
    df2 = pd.read_sql(sql.format(db_name=db_name, table_name=table_name), con=con)
df2

,CAMIS,DBA,BUILDING,STREET,ZIPCODE,BORO,PHONE,CUISINE_DESCRIPTION,LATITUDE,LONGITUDE,...,BBL,NTA,INSPECTION_DATE,ACTION,GRADE,INSPECTION_TYPE,VIOLATION_CODE,VIOLATION_DESCRIPTION,CRITICAL_FLAG,SCORE


#Insert data to DB using the to_sql command

In [27]:
# Store the dataframe as a SQL table, using the to_sql command
df.to_sql(name=table_name, # name of the table
                   con=engine, # use the connection to MySQL created earlier
                   if_exists='append', # we created the empty table above
                   index=False, # do not write the index column in the database
                   chunksize=1000 # write 1000 lines at a time
)

274388

In [28]:
# And then we can just retrieve it from the database
with engine.connect() as connection:
  r = pd.read_sql(text(f"SELECT * FROM public.{table_name} LIMIT 100"), con=connection)
r.head(100)

,CAMIS,DBA,BUILDING,STREET,ZIPCODE,BORO,PHONE,CUISINE_DESCRIPTION,LATITUDE,LONGITUDE,...,BBL,NTA,INSPECTION_DATE,ACTION,GRADE,INSPECTION_TYPE,VIOLATION_CODE,VIOLATION_DESCRIPTION,CRITICAL_FLAG,SCORE
0,50162033,WINGSTOP,2665,BROADWAY,10025,Manhattan,2125055861,None,40.7981,-73.9693,...,1018730010,MN12,1900-01-01,None,None,None,None,None,Not Applicable,NaN
1,50166191,GUAJURA ADULT DAY CARE INC,1776,BOSTON ROAD,10460,Bronx,9178171273,None,40.8379,-73.8868,...,2029910005,BX75,1900-01-01,None,None,None,None,None,Not Applicable,NaN
2,50138579,"KKOMA GIMBAB, INC.",15051,NORTHERN BLVD,11354,Queens,3476330566,None,40.7651,-73.8137,...,4050090034,QN51,1900-01-01,None,None,None,None,None,Not Applicable,NaN
3,50095998,HO HO GARDEN,9306,LIBERTY AVE,11417,Queens,7183226666,Chinese,40.6802,-73.8457,...,4091620003,QN56,2024-08-30,Establishment re-opened by DOHMH.,Z,Cycle Inspection / Reopening Inspection,None,None,Not Applicable,0.0
4,50158848,J LEVEL LLC,130,EAST 57 STREET,10022,Manhattan,9173882963,None,40.7611,-73.9695,...,1013110058,MN19,1900-01-01,None,None,None,None,None,Not Applicable,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,50154364,OASIS JIMMA JUICE BAR,342,EAST 47 STREET,10017,Manhattan,9173798330,None,40.7523,-73.9683,...,1013390027,MN19,1900-01-01,None,None,None,None,None,Not Applicable,NaN
96,50163331,9426 NEW SPRING GARDEN INC.,9426,ROOSEVELT AVE,11372,Queens,7185059918,None,40.7489,-73.8717,...,4015890096,QN29,1900-01-01,None,None,None,None,None,Not Applicable,NaN
97,41645519,NICK'S PIZZA,1356,EAST GUN HILL ROAD,10469,Bronx,7186526500,Pizza,40.8704,-73.8461,...,2045640068,BX31,2024-11-04,Violations were cited in the following area(s).,None,Cycle Inspection / Initial Inspection,02B,Hot TCS food item not held at or above 140 °F.,Critical,27.0
98,50166239,EXCELLENCE AND LONGEVITY CENTER LLC,15015,BARCLAY AVE,11355,Queens,7188657766,None,40.7618,-73.8134,...,4050587501,QN51,1900-01-01,None,None,None,None,None,Not Applicable,NaN
